In [69]:
!pip install ete3 scipy

In [87]:
import random
from scipy.stats import lognorm
from ete3 import Tree

# Define the root year
root_year = 2000

# Define the number of sequences
num_sequences = 1000

# Set the parameters of the lognormal distribution
s, loc, scale = 0.954, 0, 1

# Create the root node
t = Tree()
t.name = "root"

# Use a queue to iteratively add children to the tree
nodes_to_process = [t]
node = 0

# Keep adding nodes until we reach the required number of sequences
while len(t.get_leaves()) < num_sequences:
    
    node_to_process = random.choice(nodes_to_process)
    if len(node_to_process.get_children()) == 2:
        continue
    node+=1
    branch_length = lognorm.rvs(s, loc, scale) * 0.5
    child = node_to_process.add_child(name=f"node_{node}", dist=branch_length)
    nodes_to_process.append(child)

# merge any nodes without siblings into their parent
for node in t.traverse("postorder"):
    if node.is_leaf():
        continue
    if len(node.get_children()) == 1:
        node.get_children()[0].dist += node.dist
        node.delete()

# Write the tree to a Newick file
t.write(outfile="time_tree.nwk")


from datetime import date, timedelta

def years_to_date(years: float, start_date: date = date(2000, 1, 1)) -> date:
    return start_date + timedelta(days=years*365.25)
def assign_dates_to_nodes(tree: Tree, start_date: date = date(2000, 1, 1)):
    for node in tree.traverse("levelorder"):
        if node.is_root():
            node.add_features(date=start_date)
        else:
            elapsed_years = node.get_distance(tree)
            node_date = years_to_date(elapsed_years, start_date)
            node.add_features(date=node_date)

def write_tree_to_tsv(tree: Tree, filename: str):
    with open(filename, 'w') as f:
        f.write("strain\tdate\n")  # Write the header
        for node in tree.traverse("levelorder"):
            f.write(f"{node.name}\t{node.date}\n")

# Now call the function to create the TSV file
assign_dates_to_nodes(t)
write_tree_to_tsv(t, "tree_dates.tsv")
import os
os.system("python convert_date_file_to_lsd_format.py tree_dates.tsv > lsd_dates.tsv")

genome_size = 1000
mutations_per_site_per_year = 1e-3


old_t = t
t = t.copy()

import numpy as np

# Convert time tree to a distance tree
for node in t.traverse():
    if not node.is_root():
        # Convert branch length from years to mutations
        time_in_years = node.dist
        avg_mutations = genome_size * mutations_per_site_per_year * time_in_years
        num_mutations = np.random.poisson(avg_mutations) / genome_size

        # Update branch length to represent mutations instead of years
        node.dist = num_mutations

# Write the tree to a Newick file
t.write(outfile="distance_tree.nwk")

In [89]:

def write_tree_to_tsv(tree: Tree, filename: str):
        tuples = []
        
        for node in tree.traverse("levelorder"):
            if node.is_leaf():
                  tuples.append((node.name, node.date))
        proportion_to_randomize = 0.05
        import random
        
        with open(filename, "w") as f:
            f.write("strain\tdate\tis_randomized\n")
            for node, date in tuples:
                if random.random() < proportion_to_randomize:
                    date = tuples[random.randint(0, len(tuples)-1)][1]
                    f.write(f"{node}\t{date}\t1\n")
                else:
                    f.write(f"{node}\t{date}\t0\n")


# Now call the function to create the TSV file
assign_dates_to_nodes(old_t)
write_tree_to_tsv(old_t, "tree_dates.tsv")
os.system("python convert_date_file_to_lsd_format.py tree_dates.tsv > lsd_dates.tsv")


0

In [ ]:
!python -m chronumental --dates tree_dates.tsv --tree time_tree.nwk --treat_mutation_units_as_normalised_to_genome_size 1000 --output_unit years  --use_wandb --steps 15000 --lr 0.02

In [57]:
!treetime --dates tree_dates.tsv --tree distance_tree.nwk --sequence-length 1000 --keep-root --keep-polytomies


Attempting to parse dates...
	Using column 'strain' as name. This needs match the taxon names in the tree!!
	Using column 'date' as date.

0.00	-TreeAnc: set-up

0.16	###TreeTime.run: INITIAL ROUND

11.65	###TreeTime.run: ITERATION 1 out of 2 iterations

23.15	###TreeTime.run: CONVERGED

23.16	TreeTime: the following tips have been marked as outliers. Their date
     	constraints were not used. Please remove them from the tree. Their dates
     	have been reset:

23.16	node_2006, input date: 2000.5068306010928, apparent date: 2010.94

Inferred sequence evolution model (saved as 2023-08-01_treetime/sequence_evolution_model.txt):
Substitution rate (mu): 1.0

Equilibrium frequencies (pi_i):
  A: 0.2
  C: 0.2
  G: 0.2
  T: 0.2
  -: 0.2

Symmetrized rates from j->i (W_ij):
	A	C	G	T	-
  A	0	1.25	1.25	1.25	1.25
  C	1.25	0	1.25	1.25	1.25
  G	1.25	1.25	0	1.25	1.25
  T	1.25	1.25	1.25	0	1.25
  -	1.25	1.25	1.25	1.25	0

Actual rates from j->i (Q_ij):
	A	C	G	T	-
  A	0	0.25	0.25	0.25	0.25
  C	0.25	0

In [61]:
!python compare.py  2023-08-01_treetime/timetree.nexus time_tree.nwk

((node_1352:0.56902)NODE_0000001:1.04774,((((node_1549:2.93087)NODE_0000005:0.00000)NODE_0000004:0.00000)NODE_0000003:1.89819,node_1849:0.00000,node_1934:0.34426)NODE_0000002:0.94934,((node_764:3.23547,(node_1463:0.00000)NODE_0000008:3.10671)NODE_0000007:0.00000,((node_740:1.55412)NODE_0000010:0.75807,(node_1054:0.55307)NODE_0000011:1.12350,(((node_1725:5.14749)NODE_0000014:0.00000)NODE_0000013:1.78113,((node_1956:5.11245)NODE_0000016:0.00000,node_1965:1.68505)NODE_0000015:0.67097)NODE_0000012:0.00000)NODE_0000009:0.00000)NODE_0000006:0.95663,(node_1742:1.27175,node_1868:0.91109,(node_78:0.49046,(node_1806:0.00000)NODE_0000019:2.52608,((node_1496:0.00000)NODE_0000021:0.95895,node_1169:0.77539,(node_1981:0.00000)NODE_0000022:1.95347,(node_1024:0.14521,node_1303:0.00000)NODE_0000023:1.58635,(node_963:1.27676,(node_1892:1.29320)NODE_0000025:0.00000,(node_1979:1.68772)NODE_0000026:0.00000)NODE_0000024:0.00000,(((node_1576:0.00789)NODE_0000029:0.00099)NODE_0000028:1.66579,(node_1826:0.00000

In [ ]:
!python -m chronumental --dates tree_dates.tsv --tree time_tree.nwk --treat_mutation_units_as_normalised_to_genome_size 1000 --output_unit years  --use_wandb --model HorseShoeLike



In [60]:
!python compare.py  chronumental_timetree_time_tree.nwk time_tree.nwk

2.1979212434734365,-1,-1


In [90]:
!python compare.py  distance_tree.nwk.result.nexus time_tree.nwk

 ((((((((((node_1820:0.000176956,node_2108:0.00239501)1:0.00157197,node_1855:-0.000500371)1:0.0020716,node_350:0.000274756)1:0.000346351,(((((node_1607:0.00162735,node_2516:0.00200223)1:-0.000370419,node_1939:0.000741467)1:-0.000628952,(((node_2457:0.0014647,node_2475:0.00226132)1:-0.000273984,node_1493:0.00180771)1:0.000533721,((node_1576:0.000621172,node_2067:0.00162867)1:-0.00075016,(node_2591:-6.90297e-05,node_2541:0.000118411)1:4.93817e-05)1:0.000299221)1:-0.000167058)1:0.000203991,node_1412:0.00244865)1:0.000652641,(((node_1124:0.000324068,node_1170:0.000136627)1:0.0014607,(((((node_2127:-0.000146891,node_2603:0.00378156)1:0.000634672,(node_2194:0.000397,node_2612:0.00109209)1:0.000489094)1:0.00112377,node_2138:0.0025097)1:0.000633469,node_2208:0.00361958)1:0.000253054,(((node_1767:0.00199253,node_1885:0.0017348)1:-0.000272677,node_1420:0.000610823)1:0.000338145,((node_2249:0.000858,node_2600:0.00234972)1:-0.000792281,node_2433:-0.000121722)1:0.002086)1:0.00142414)1:-0.000322804)

In [ ]:
! ~/Desktop/lsd-0.2/src/lsd -i distance_tree.nwk -d lsd_dates.tsv